In [1]:
import numpy as np 
import tensorflow as tf 
import pandas as pd
import os
import utils

%matplotlib inline

!wandb login "41c25b4fc8e96d4ae0d96e0abd4d69787a6ea35f"
import wandb
from wandb.keras import WandbCallback
wandb.init(project="lstm-autoencoder-esc190")
config = wandb.config


Successfully logged in to Weights & Biases!
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\DS/.netrc


In [2]:
data_dir = "data_wordnet"
data = utils.read_dir(data_dir)
definitions, max_length, wordmap = utils.get_definitions(data)

print("Size of vocabulary: {}".format(len(data)))
print("Longest definition (words): {}".format(max_length))
print("Number of definitions: {}".format(len(definitions)))

word2num, num2word = utils.get_word_dicts(definitions)
vocab_size = len(list(word2num.keys()))

print("Size of definition vocabulary: {}".format(vocab_size))

def_vectors = utils.convert_word2int(definitions, word2num)

x_train = utils.defs_to_np(def_vectors, max_length)

print(x_train.shape)

Size of vocabulary: 117659
Longest definition (words): 54
Number of definitions: 135959
Size of definition vocabulary: 46948
(135959, 54)


In [14]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Input, Embedding, LSTM, RepeatVector, TimeDistributed, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy

'''
class LSTMAutoencoder(Model):
    def __init__(self, vocab_size, max_length):
        super(LSTMAutoencoder, self).__init__()
        #self.inputy = Input(shape=(None,), dtype="int32")
        self.embedding = Embedding(input_dim=50000, output_dim=64, input_length=max_length, mask_zero=True)
        self.encodingLSTM1 = LSTM(32, return_sequences=True)
        self.encodingLSTM2 = LSTM(16)
        self.repeatlayer = RepeatVector(max_length)
        self.decodingLSTM1 = LSTM(16, return_sequences=True)
        self.decodingLSTM2 = LSTM(32, return_sequences=True)
        self.denseboi = TimeDistributed(Dense(100, activation="relu"))
        self.finalDense = TimeDistributed(Dense(vocab_size, activation="softmax"))

    def call(self, inputs):
        #x = self.inputy(inputs)
        
        x = self.embedding(inputs)
        mask = self.embedding.compute_mask(inputs)
        x = self.encodingLSTM1(x, mask=mask)
        x = self.encodingLSTM2(x, mask=mask)
        x = self.repeatlayer(x)
        x = self.decodingLSTM1(x, mask=mask)
        x = self.decodingLSTM2(x, mask=mask)
        x = self.denseboi(x)
        x = self.finalDense(x)
        print(x.shape)
        return x

'''
#training embedding layer
'''
embedding = Sequential()
embedding.add(Embedding(input_dim=vocab_size+1, output_dim=64, input_length=max_length, mask_zero=True))
'''
'''
inputs = Input(shape=(None,))
embedding = Embedding(input_dim=vocab_size+1, output_dim=64, input_length=max_length, mask_zero=True)(inputs)
mask = Embedding(input_dim=50000, output_dim=64, input_length=max_length, mask_zero=True).compute_mask(inputs)
encodingLSTM1 = LSTM(32, return_sequences=True)(embedding, mask=mask)
encodingLSTM2 = LSTM(16)(encodingLSTM1, mask=mask)
repeatlayer = RepeatVector(max_length)(encodingLSTM2)
decodingLSTM1 = LSTM(16, return_sequences=True)(repeatlayer, mask=mask)
decodingLSTM2 = LSTM(32, return_sequences=True)(decodingLSTM1, mask=mask)
denseboi = TimeDistributed(Dense(100, activation="relu"))(decodingLSTM2)
finalDense = TimeDistributed(Dense(vocab_size, activation="softmax"))(denseboi)
output = finalDense
'''

inputs = Input(shape=(None,))
embedding = Embedding(input_dim=vocab_size+1, output_dim=64, input_length=max_length, mask_zero=True)(inputs)
mask = Embedding(input_dim=50000, output_dim=64, input_length=max_length, mask_zero=True).compute_mask(inputs)
encodingLSTM1 = LSTM(32, return_sequences=True)(embedding, mask=mask)
encodingLSTM2 = LSTM(16)(encodingLSTM1, mask=mask)
repeatlayer = RepeatVector(max_length)(encodingLSTM2)
decodingLSTM1 = LSTM(16, return_sequences=True)(repeatlayer, mask=mask)
decodingLSTM2 = LSTM(32, return_sequences=True)(decodingLSTM1, mask=mask)
denseboi = TimeDistributed(Dense(100, activation="relu"))(decodingLSTM2)
finalDense = TimeDistributed(Dense(vocab_size, activation="softmax"))(denseboi)
output = finalDense
#creating optimizers and loss

In [4]:
model = Model(inputs=inputs, outputs=output)
optimizer = Adam(learning_rate = 0.0003)
model.compile(loss = categorical_crossentropy, optimizer = optimizer, metrics = ["accuracy"])

#embedding.compile("rmsprop", "mse")

model.summary()
#embedding.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 64)     3004736     input_1[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_NotEqual (TensorFlo [(None, None)]       0           input_1[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     (None, None, 32)     12416       embedding[0][0]                  
______________________________________________________________________________________________

In [ ]:
config.batch_size = 16
#config.steps_per_epoch = 100
config.epochs = 8

In [ ]:
x_train = x_train.astype("int32")
np.random.shuffle(x_train)
dataset = tf.data.Dataset.from_tensor_slices(x_train)
dataset = dataset.map(utils.to_one_hot)
dataset = dataset.shuffle(1000).batch(config.batch_size)

In [7]:
model.load_weights("16vec.weights.h5")

NameError: name 'model' is not defined

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
save_model = ModelCheckpoint(filepath="32vec.5.weights.{epoch:02d}.h5", monitor='accuracy', save_weights_only=True, mode='auto', verbose=1)

In [ ]:
wandb.init()
model.fit(dataset, epochs=config.epochs, callbacks=[WandbCallback(), save_model])

In [15]:
#creating dictionary of vectors
encoding_model = Model(inputs=inputs, outputs=encodingLSTM2)


optimizer = Adam(learning_rate = 0.0003)
encoding_model.compile(loss = categorical_crossentropy, optimizer = optimizer, metrics = ["accuracy"])

encoding_model.load_weights("32vec.weights.h5", by_name=True)
encoding_model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, None, 64)     3004736     input_4[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_NotEqual_1 (TensorF [(None, None)]       0           input_4[0][0]                    
__________________________________________________________________________________________________
lstm_12 (LSTM)                  (None, None, 64)     33024       embedding_4[0][0]                
____________________________________________________________________________________________

In [16]:
output = encoding_model.predict(x_train)


UnknownError:  [_Derived_]  Fail to find the dnn implementation.
	 [[{{node cond/then/_0/CudnnRNNV3}}]]
	 [[model_4/lstm_12/StatefulPartitionedCall]] [Op:__inference_distributed_function_33118]

Function call stack:
distributed_function -> distributed_function -> distributed_function


In [30]:


word2vecs = {}

for index, word in enumerate(wordmap):
    if word in word2vecs:
        word2vecs[word].append(output[index, :])
    else:
        word2vecs[word] = [output[index, :]]
vec2word = {tuple(key[0]): value for value, key in word2vecs.items()}

In [28]:
import pickle

pickle.dump(vec2word, open("vec2word32", "wb"))
pickle.dump(word2vecs, open("word2vecs32", "wb"))

In [33]:
print(len(list(word2vecs.values())))
print(len(list(vec2word.values())))

87632
86430
